In [1]:
import pandas as pd

In [60]:
pd.set_option('display.max_rows', pd.get_option('display.max_rows'))

**IMPORT SURVEY DATABASE

In [2]:
file_path = r"C:\Users\AaronH\Documents\nss_data_analytics\Projects\federal_employee_job_satisfaction\data\SurveyData.csv"

In [3]:
survey = pd.read_csv(file_path, header=None)

C:\Users\AaronH\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,2,3,4,5,6,7,8,10,11,12,14,15,16,17,18,19,20,21,30,31,34,35,44,49,53,54,55,56,57,58,59,71,72,73,74,75,76,91,92,93,94,95,96,97,98,99,101,115) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
survey_table = survey

In [5]:
survey_table

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,RandomID,agency,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,...,DRNO,DHISP,DDIS,DAGEGRP,DSUPER,DFEDTEN,DSEX,DMIL,DLEAVING,POSTWT
1,1.12971E+11,XX,4,4,4,4,2,4,4,4,...,B,B,B,B,B,B,A,A,A,2.238814299
2,1.94869E+11,XX,4,3,5,4,4,4,5,4,...,B,B,B,B,A,A,A,A,A,2.238814299
3,1.52966E+11,XX,5,5,5,5,2,5,5,2,...,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,1.502352285
4,1.93041E+11,XX,2,2,4,1,3,2,2,1,...,B,B,B,B,A,A,B,A,A,1.756431349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557774,1.9578e+11,ST,4,4,5,4,3,4,4,3,...,B,B,B,B,B,C,A,B,B,2.48538
557775,1.02163e+11,ST,4,4,4,4,2,3,3,4,...,B,B,B,B,B,C,A,B,D,1.67791
557776,1.64068e+11,ST,5,5,4,4,3,5,5,5,...,B,B,B,B,B,C,A,B,B,3.58316
557777,1.55255e+11,ST,5,5,5,5,4,5,5,5,...,B,B,B,B,B,C,B,B,A,2.73418


In [6]:
survey_table.columns = survey_table.iloc[0]

In [7]:
survey_table = survey_table[1:]

In [8]:
survey_table

,RandomID,agency,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,...,DRNO,DHISP,DDIS,DAGEGRP,DSUPER,DFEDTEN,DSEX,DMIL,DLEAVING,POSTWT
1,1.12971E+11,XX,4,4,4,4,2,4,4,4,...,B,B,B,B,B,B,A,A,A,2.238814299
2,1.94869E+11,XX,4,3,5,4,4,4,5,4,...,B,B,B,B,A,A,A,A,A,2.238814299
3,1.52966E+11,XX,5,5,5,5,2,5,5,2,...,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,1.502352285
4,1.93041E+11,XX,2,2,4,1,3,2,2,1,...,B,B,B,B,A,A,B,A,A,1.756431349
5,1.46656E+11,XX,4,4,4,4,4,2,4,2,...,B,B,B,A,B,A,B,B,A,1.756431349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557774,1.9578e+11,ST,4,4,5,4,3,4,4,3,...,B,B,B,B,B,C,A,B,B,2.48538
557775,1.02163e+11,ST,4,4,4,4,2,3,3,4,...,B,B,B,B,B,C,A,B,D,1.67791
557776,1.64068e+11,ST,5,5,4,4,3,5,5,5,...,B,B,B,B,B,C,A,B,B,3.58316
557777,1.55255e+11,ST,5,5,5,5,4,5,5,5,...,B,B,B,B,B,C,B,B,A,2.73418


**REMAP AGENCY NAME

In [9]:
agency_map = {'AF': 'United States Department of the Air Force', 'AG': 'Department of Agriculture', 'AM':'U.S. Agency for International Development', 
             'AR':'United States Department of the Army','CM':'Department of Commerce','CU':'National Credit Union Administration','DD':'OSD, Joint Staff, Defense Agencies, and Field Activities',
             'DJ':'Department of Justice','DL':'Department of Labor','DN':'Department of Energy','DR':'Federal Energy Regulatory Commission',
             'ED':'Department of Education','EE':'Equal Employment Opportunity Commission','EP':'Environmental Protection Agency','FT':'Federal Trade Commission',
             'GS':'General Services Administration','HE':'Department of Health and Human Services','HS':'Department of Homeland Security',
             'HU':'Department of Housing and Urban Development','IN':'Department of the Interior','NF':'National Science Foundation','NQ':'National Archives and Records Administration',
             'NU':'Nuclear Regulatory Commission','NV':'United States Department of the Navy','OM':'Office of Personnel Management','SB':'Small Business Administration',
             'ST':'Department of State','SZ':'Social Security Administration','TD':'Department of Transportation','TR':'Department of the Treasury','XX':'All Other Agencies'}

In [10]:
survey_table['AGENCY'] = survey_table['agency'].map(agency_map)

C:\Users\AaronH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
groupby_survey = survey_table.groupby('AGENCY').size().reset_index()

In [46]:
groupby_survey.to_clipboard()

In [105]:
survey_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557778 entries, 1 to 557778
Columns: 118 entries, RandomID to my_work_experience_index
dtypes: float64(1), object(117)
memory usage: 502.2+ MB


**CREATE INDEX SCORE COLUMN LISTS

*My Work Experience

In [106]:
my_work_experience =['Q1', 'Q2', 'Q3', 'Q4','Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13']

In [107]:
survey_table[my_work_experience] = survey_table[my_work_experience].replace('X', pd.NA)

C:\Users\AaronH\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [110]:
survey_table[my_work_experience] = survey_table[my_work_experience].apply(pd.to_numeric, errors='coerce')

C:\Users\AaronH\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [111]:
survey_table['my_work_experience_index'] = survey_table[my_work_experience].mean(axis=1)

C:\Users\AaronH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [112]:
survey_table

,RandomID,agency,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,...,DDIS,DAGEGRP,DSUPER,DFEDTEN,DSEX,DMIL,DLEAVING,POSTWT,AGENCY,my_work_experience_index
1,1.12971E+11,XX,4.0,4.0,4.0,4.0,2.0,4.0,4.0,4.0,...,B,B,B,B,A,A,A,2.238814299,All Other Agencies,3.846154
2,1.94869E+11,XX,4.0,3.0,5.0,4.0,4.0,4.0,5.0,4.0,...,B,B,A,A,A,A,A,2.238814299,All Other Agencies,3.923077
3,1.52966E+11,XX,5.0,5.0,5.0,5.0,2.0,5.0,5.0,2.0,...,B,NaN,NaN,NaN,NaN,NaN,NaN,1.502352285,All Other Agencies,4.076923
4,1.93041E+11,XX,2.0,2.0,4.0,1.0,3.0,2.0,2.0,1.0,...,B,B,A,A,B,A,A,1.756431349,All Other Agencies,2.615385
5,1.46656E+11,XX,4.0,4.0,4.0,4.0,4.0,2.0,4.0,2.0,...,B,A,B,A,B,B,A,1.756431349,All Other Agencies,3.538462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557774,1.9578e+11,ST,4.0,4.0,5.0,4.0,3.0,4.0,4.0,3.0,...,B,B,B,C,A,B,B,2.48538,Department of State,3.461538
557775,1.02163e+11,ST,4.0,4.0,4.0,4.0,2.0,3.0,3.0,4.0,...,B,B,B,C,A,B,D,1.67791,Department of State,3.384615
557776,1.64068e+11,ST,5.0,5.0,4.0,4.0,3.0,5.0,5.0,5.0,...,B,B,B,C,A,B,B,3.58316,Department of State,4.230769
557777,1.55255e+11,ST,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,...,B,B,B,C,B,B,A,2.73418,Department of State,4.461538


*My Work Unit

In [93]:
my_work_unit =['Q14', 'Q16', 'Q17', 'Q18','Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28', 'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34']

In [94]:
my_organization = ['Q35', 'Q36', 'Q37', 'Q38','Q39', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44']

In [95]:
supervisor = ['Q45', 'Q46', 'Q47', 'Q48','Q49', 'Q50', 'Q51', 'Q52', 'Q53', 'Q54']

In [96]:
leadership = ['Q55', 'Q56', 'Q57', 'Q58','Q59', 'Q60', 'Q61', 'Q62', 'Q63', 'Q64']

In [97]:
my_satisfaction = ['Q65', 'Q66', 'Q67', 'Q68','Q69', 'Q70']

In [98]:
dei =['Q71', 'Q72', 'Q73', 'Q74','Q75', 'Q76', 'Q77', 'Q78', 'Q79', 'Q80', 'Q81', 'Q82', 'Q83', 'Q84']

In [99]:
employee_experience = ['Q85', 'Q86', 'Q87', 'Q88','Q89']

**CREATE INDEX SCORE COLUMN AVERAGES

In [108]:
survey_table['my_work_experience_index'] = survey_table[my_work_experience].mean(axis=1)

C:\Users\AaronH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [109]:
survey_table

,RandomID,agency,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,...,DDIS,DAGEGRP,DSUPER,DFEDTEN,DSEX,DMIL,DLEAVING,POSTWT,AGENCY,my_work_experience_index
1,1.12971E+11,XX,4,4,4,4,2,4,4,4,...,B,B,B,B,A,A,A,2.238814299,All Other Agencies,NaN
2,1.94869E+11,XX,4,3,5,4,4,4,5,4,...,B,B,A,A,A,A,A,2.238814299,All Other Agencies,NaN
3,1.52966E+11,XX,5,5,5,5,2,5,5,2,...,B,NaN,NaN,NaN,NaN,NaN,NaN,1.502352285,All Other Agencies,NaN
4,1.93041E+11,XX,2,2,4,1,3,2,2,1,...,B,B,A,A,B,A,A,1.756431349,All Other Agencies,NaN
5,1.46656E+11,XX,4,4,4,4,4,2,4,2,...,B,A,B,A,B,B,A,1.756431349,All Other Agencies,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557774,1.9578e+11,ST,4,4,5,4,3,4,4,3,...,B,B,B,C,A,B,B,2.48538,Department of State,NaN
557775,1.02163e+11,ST,4,4,4,4,2,3,3,4,...,B,B,B,C,A,B,D,1.67791,Department of State,NaN
557776,1.64068e+11,ST,5,5,4,4,3,5,5,5,...,B,B,B,C,A,B,B,3.58316,Department of State,NaN
557777,1.55255e+11,ST,5,5,5,5,4,5,5,5,...,B,B,B,C,B,B,A,2.73418,Department of State,NaN


**IMPORT EMPLOYMENT DATABASE

In [12]:
file_path = r"C:\Users\AaronH\Documents\nss_data_analytics\Projects\nss-capstone\data\FACTDATA_SEP2023.TXT"

In [13]:
employment = pd.read_csv(file_path, header=None)

C:\Users\AaronH\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,4,6,7,8,12,13,15,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
employment

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,SALLVL,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS
1,AA00,11,F,13,NaN,G,0340,2,ES,ES-**,T,XXXX,2,50,F,1,202309,1,210000,20.3
2,AA00,11,I,15,NaN,I,0905,1,ES,ES-**,T,XXXX,2,50,F,1,202309,1,203000,30.7
3,AA00,11,F,04,14,E,0905,1,99,GS-14,Q,XXXX,8,30,F,1,202309,1,172075,14.1
4,AA00,11,E,15,NaN,E,0905,1,ES,ES-**,S,XXXX,8,50,F,1,202309,1,195000,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258817,ZS00,11,C,4,NaN,B,0301,2,99,AD-00,F,XXXX,8,48,F,2,202309,1,67500,2.5
2258818,ZS00,11,D,4,NaN,A,0301,2,99,AD-00,R,XXXX,8,48,I,2,202309,1,184115,0.7
2258819,ZS00,11,D,4,NaN,A,0301,2,99,AD-00,E,XXXX,8,48,F,2,202309,1,50000,0.4
2258820,ZS00,11,C,13,NaN,A,0301,2,99,AD-00,E,XXXX,8,48,F,2,202309,1,56000,0.4


In [68]:
employment_table = employment

In [69]:
employment_table.columns = employment_table.iloc[0]

In [70]:
employment_table = employment_table[1:]

In [71]:
employment_table

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,SALLVL,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS
1,AA00,11,F,13,NaN,G,0340,2,ES,ES-**,T,XXXX,2,50,F,1,202309,1,210000,20.3
2,AA00,11,I,15,NaN,I,0905,1,ES,ES-**,T,XXXX,2,50,F,1,202309,1,203000,30.7
3,AA00,11,F,04,14,E,0905,1,99,GS-14,Q,XXXX,8,30,F,1,202309,1,172075,14.1
4,AA00,11,E,15,NaN,E,0905,1,ES,ES-**,S,XXXX,8,50,F,1,202309,1,195000,12.0
5,AA00,11,G,15,14,E,0905,1,99,GS-14,O,XXXX,8,30,F,1,202309,1,150016,11.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258817,ZS00,11,C,4,NaN,B,0301,2,99,AD-00,F,XXXX,8,48,F,2,202309,1,67500,2.5
2258818,ZS00,11,D,4,NaN,A,0301,2,99,AD-00,R,XXXX,8,48,I,2,202309,1,184115,0.7
2258819,ZS00,11,D,4,NaN,A,0301,2,99,AD-00,E,XXXX,8,48,F,2,202309,1,50000,0.4
2258820,ZS00,11,C,13,NaN,A,0301,2,99,AD-00,E,XXXX,8,48,F,2,202309,1,56000,0.4


In [72]:
employment_table['agency'] = employment_table['AGYSUB'].apply(lambda x: x[:2] if x != 'DNFE' else x)

C:\Users\AaronH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [73]:
employment_table

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,...,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS,agency
1,AA00,11,F,13,NaN,G,0340,2,ES,ES-**,...,XXXX,2,50,F,1,202309,1,210000,20.3,AA
2,AA00,11,I,15,NaN,I,0905,1,ES,ES-**,...,XXXX,2,50,F,1,202309,1,203000,30.7,AA
3,AA00,11,F,04,14,E,0905,1,99,GS-14,...,XXXX,8,30,F,1,202309,1,172075,14.1,AA
4,AA00,11,E,15,NaN,E,0905,1,ES,ES-**,...,XXXX,8,50,F,1,202309,1,195000,12.0,AA
5,AA00,11,G,15,14,E,0905,1,99,GS-14,...,XXXX,8,30,F,1,202309,1,150016,11.3,AA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258817,ZS00,11,C,4,NaN,B,0301,2,99,AD-00,...,XXXX,8,48,F,2,202309,1,67500,2.5,ZS
2258818,ZS00,11,D,4,NaN,A,0301,2,99,AD-00,...,XXXX,8,48,I,2,202309,1,184115,0.7,ZS
2258819,ZS00,11,D,4,NaN,A,0301,2,99,AD-00,...,XXXX,8,48,F,2,202309,1,50000,0.4,ZS
2258820,ZS00,11,C,13,NaN,A,0301,2,99,AD-00,...,XXXX,8,48,F,2,202309,1,56000,0.4,ZS


In [74]:
agency_map = {'AF': 'United States Department of the Air Force', 'AG': 'Department of Agriculture', 'AM':'U.S. Agency for International Development', 
             'AR':'United States Department of the Army','CM':'Department of Commerce','CU':'National Credit Union Administration','DD':'OSD, Joint Staff, Defense Agencies, and Field Activities',
             'DJ':'Department of Justice','DL':'Department of Labor','DN':'Department of Energy', 'DNFE':'Federal Energy Regulatory Commission', 'DR':'Federal Energy Regulatory Commission',
             'ED':'Department of Education','EE':'Equal Employment Opportunity Commission','EP':'Environmental Protection Agency','FT':'Federal Trade Commission',
             'GS':'General Services Administration','HE':'Department of Health and Human Services','HS':'Department of Homeland Security',
             'HU':'Department of Housing and Urban Development','IN':'Department of the Interior','NF':'National Science Foundation','NQ':'National Archives and Records Administration',
             'NU':'Nuclear Regulatory Commission','NV':'United States Department of the Navy','OM':'Office of Personnel Management','SB':'Small Business Administration',
             'ST':'Department of State','SZ':'Social Security Administration','TD':'Department of Transportation','TR':'Department of the Treasury','XX':'All Other Agencies'}

In [75]:
employment_table['AGENCY'] = employment_table['agency'].map(agency_map)

C:\Users\AaronH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [76]:
employment_table

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,...,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS,agency,AGENCY
1,AA00,11,F,13,NaN,G,0340,2,ES,ES-**,...,2,50,F,1,202309,1,210000,20.3,AA,NaN
2,AA00,11,I,15,NaN,I,0905,1,ES,ES-**,...,2,50,F,1,202309,1,203000,30.7,AA,NaN
3,AA00,11,F,04,14,E,0905,1,99,GS-14,...,8,30,F,1,202309,1,172075,14.1,AA,NaN
4,AA00,11,E,15,NaN,E,0905,1,ES,ES-**,...,8,50,F,1,202309,1,195000,12.0,AA,NaN
5,AA00,11,G,15,14,E,0905,1,99,GS-14,...,8,30,F,1,202309,1,150016,11.3,AA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258817,ZS00,11,C,4,NaN,B,0301,2,99,AD-00,...,8,48,F,2,202309,1,67500,2.5,ZS,NaN
2258818,ZS00,11,D,4,NaN,A,0301,2,99,AD-00,...,8,48,I,2,202309,1,184115,0.7,ZS,NaN
2258819,ZS00,11,D,4,NaN,A,0301,2,99,AD-00,...,8,48,F,2,202309,1,50000,0.4,ZS,NaN
2258820,ZS00,11,C,13,NaN,A,0301,2,99,AD-00,...,8,48,F,2,202309,1,56000,0.4,ZS,NaN


In [77]:
employment_table['AGENCY'] = employment_table['AGENCY'].fillna('missing') 

C:\Users\AaronH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
groupby = employment_table.groupby('AGENCY').size().reset_index()

In [79]:
groupby

,AGENCY,0
0,Department of Agriculture,95248
1,Department of Commerce,49020
2,Department of Education,4252
3,Department of Energy,14801
4,Department of Health and Human Services,90337
5,Department of Homeland Security,219092
6,Department of Housing and Urban Development,8559
7,Department of Justice,116390
8,Department of Labor,15108
9,Department of State,13814


In [80]:
groupby.to_clipboard()

**IMPORT SEPARATIONS DATABASE

In [3]:
file_path = r"C:\Users\AaronH\Documents\nss_data_analytics\Projects\nss-capstone\data\SeparationsData.xlsx"

In [6]:
separations = pd.read_excel(file_path, header=None)

In [7]:
separations_table = separations[0].str.split(',', expand=True)

In [8]:
separations_table

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,AGYSUB,SEP,EFDATE,AGELVL,EDLVL,GSEGRD,LOSLVL,LOC,OCC,PATCO,PPGRD,SALLVL,STEMOCC,TOA,WORKSCH,WORKSTAT,COUNT,SALARY,LOS
1,AA00,SJ,202006,J,18,14,E,11,0905,1,GS-14,L,XXXX,40,F,2,1,125360,12.3
2,AA00,SC,202208,F,15,15,E,51,0905,1,GS-15,Q,XXXX,30,F,1,1,176300,12.9
3,AA00,SC,202207,H,15,,E,11,0301,2,ES-**,S,XXXX,50,F,1,1,199300,12.8
4,AA00,SA,202008,D,14,14,C,11,0905,1,GS-14,L,XXXX,30,F,1,1,121316,04.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717108,ZU00,SA,202111,K,17,,E,11,0301,2,AD-00,Q,XXXX,48,F,2,1,170800,14.5
717109,ZU00,SJ,202107,E,17,,D,11,0301,2,AD-00,K,XXXX,38,F,1,1,112930,08.1
717110,ZU00,SA,202103,K,13,,D,11,0301,2,AD-00,M,XXXX,38,F,1,1,133465,08.0
717111,ZU00,SJ,202107,C,13,,B,11,0301,2,AD-00,F,XXXX,38,F,1,1,064682,02.8


In [11]:
separations_table.columns = separations_table.iloc[0]

In [12]:
separations_table

,AGYSUB,SEP,EFDATE,AGELVL,EDLVL,GSEGRD,LOSLVL,LOC,OCC,PATCO,PPGRD,SALLVL,STEMOCC,TOA,WORKSCH,WORKSTAT,COUNT,SALARY,LOS
0,AGYSUB,SEP,EFDATE,AGELVL,EDLVL,GSEGRD,LOSLVL,LOC,OCC,PATCO,PPGRD,SALLVL,STEMOCC,TOA,WORKSCH,WORKSTAT,COUNT,SALARY,LOS
1,AA00,SJ,202006,J,18,14,E,11,0905,1,GS-14,L,XXXX,40,F,2,1,125360,12.3
2,AA00,SC,202208,F,15,15,E,51,0905,1,GS-15,Q,XXXX,30,F,1,1,176300,12.9
3,AA00,SC,202207,H,15,,E,11,0301,2,ES-**,S,XXXX,50,F,1,1,199300,12.8
4,AA00,SA,202008,D,14,14,C,11,0905,1,GS-14,L,XXXX,30,F,1,1,121316,04.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717108,ZU00,SA,202111,K,17,,E,11,0301,2,AD-00,Q,XXXX,48,F,2,1,170800,14.5
717109,ZU00,SJ,202107,E,17,,D,11,0301,2,AD-00,K,XXXX,38,F,1,1,112930,08.1
717110,ZU00,SA,202103,K,13,,D,11,0301,2,AD-00,M,XXXX,38,F,1,1,133465,08.0
717111,ZU00,SJ,202107,C,13,,B,11,0301,2,AD-00,F,XXXX,38,F,1,1,064682,02.8


In [13]:
separations_table = separations_table[1:]

In [14]:
separations_table

,AGYSUB,SEP,EFDATE,AGELVL,EDLVL,GSEGRD,LOSLVL,LOC,OCC,PATCO,PPGRD,SALLVL,STEMOCC,TOA,WORKSCH,WORKSTAT,COUNT,SALARY,LOS
1,AA00,SJ,202006,J,18,14,E,11,0905,1,GS-14,L,XXXX,40,F,2,1,125360,12.3
2,AA00,SC,202208,F,15,15,E,51,0905,1,GS-15,Q,XXXX,30,F,1,1,176300,12.9
3,AA00,SC,202207,H,15,,E,11,0301,2,ES-**,S,XXXX,50,F,1,1,199300,12.8
4,AA00,SA,202008,D,14,14,C,11,0905,1,GS-14,L,XXXX,30,F,1,1,121316,04.9
5,AA00,SA,202003,D,15,14,D,11,0905,1,GS-14,N,XXXX,40,F,2,1,149621,05.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717108,ZU00,SA,202111,K,17,,E,11,0301,2,AD-00,Q,XXXX,48,F,2,1,170800,14.5
717109,ZU00,SJ,202107,E,17,,D,11,0301,2,AD-00,K,XXXX,38,F,1,1,112930,08.1
717110,ZU00,SA,202103,K,13,,D,11,0301,2,AD-00,M,XXXX,38,F,1,1,133465,08.0
717111,ZU00,SJ,202107,C,13,,B,11,0301,2,AD-00,F,XXXX,38,F,1,1,064682,02.8


In [25]:
separations_table['DATE'] = pd.to_datetime(separations_table['EFDATE'], format='%Y%m')

C:\Users\AaronH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
separations_table['DATE'] = separations_table['DATE'].dt.to_period('M')

C:\Users\AaronH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
separations_table

,AGYSUB,SEP,EFDATE,AGELVL,EDLVL,GSEGRD,LOSLVL,LOC,OCC,PATCO,...,STEMOCC,TOA,WORKSCH,WORKSTAT,COUNT,SALARY,LOS,DATE,YEAR,MONTH
1,AA00,SJ,202006,J,18,14,E,11,0905,1,...,XXXX,40,F,2,1,125360,12.3,2020-06,2020,6
2,AA00,SC,202208,F,15,15,E,51,0905,1,...,XXXX,30,F,1,1,176300,12.9,2022-08,2022,8
3,AA00,SC,202207,H,15,,E,11,0301,2,...,XXXX,50,F,1,1,199300,12.8,2022-07,2022,7
4,AA00,SA,202008,D,14,14,C,11,0905,1,...,XXXX,30,F,1,1,121316,04.9,2020-08,2020,8
5,AA00,SA,202003,D,15,14,D,11,0905,1,...,XXXX,40,F,2,1,149621,05.3,2020-03,2020,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717108,ZU00,SA,202111,K,17,,E,11,0301,2,...,XXXX,48,F,2,1,170800,14.5,2021-11,2021,11
717109,ZU00,SJ,202107,E,17,,D,11,0301,2,...,XXXX,38,F,1,1,112930,08.1,2021-07,2021,7
717110,ZU00,SA,202103,K,13,,D,11,0301,2,...,XXXX,38,F,1,1,133465,08.0,2021-03,2021,3
717111,ZU00,SJ,202107,C,13,,B,11,0301,2,...,XXXX,38,F,1,1,064682,02.8,2021-07,2021,7


In [52]:
pd.set_option('display.max_rows', 500)

In [44]:
sep = separations_table.groupby('AGYSUB').size()

In [45]:
sep

AGYSUB
AA00         9
AB00        39
AF**         3
AF02        18
AF03        79
AF06       133
AF07       237
AF09       388
AF0B       809
AF0D      1274
AF0J      6763
AF0M      3875
AF0R      2153
AF0V       604
AF12         8
AF13       877
AF16         9
AF17       271
AF1B        43
AF1C      4468
AF1L      3153
AF1M     16059
AF1Q        25
AF1S       512
AF1Y       823
AF20         9
AF2A        18
AF2E        53
AF2F        13
AF2H         6
AF2I       237
AF2K         7
AF2L       117
AF2M        23
AF2R         7
AF2S        35
AF2T        20
AF2Z       119
AF34        10
AF38        36
AF3C       132
AF3D       350
AF3Q       455
AF3T       276
AF3V        90
AF40       392
AF4C       116
AF4D       265
AF4F        27
AF4G      1300
AF4S        73
AF4V       234
AF4W       681
AF5D        23
AF6F       383
AFGS      1771
AFNG     17030
AFZG       228
AG01        97
AG02      1436
AG03      3536
AG07      1250
AG08        71
AG10       315
AG11     34210
AG13        14
AG1

In [50]:
separations_table['AGENCY'] = separations_table['AGYSUB'].str[:2]

C:\Users\AaronH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
separations_table

,AGYSUB,SEP,EFDATE,AGELVL,EDLVL,GSEGRD,LOSLVL,LOC,OCC,PATCO,...,TOA,WORKSCH,WORKSTAT,COUNT,SALARY,LOS,DATE,YEAR,MONTH,AGENCY
1,AA00,SJ,202006,J,18,14,E,11,0905,1,...,40,F,2,1,125360,12.3,2020-06,2020,6,AA
2,AA00,SC,202208,F,15,15,E,51,0905,1,...,30,F,1,1,176300,12.9,2022-08,2022,8,AA
3,AA00,SC,202207,H,15,,E,11,0301,2,...,50,F,1,1,199300,12.8,2022-07,2022,7,AA
4,AA00,SA,202008,D,14,14,C,11,0905,1,...,30,F,1,1,121316,04.9,2020-08,2020,8,AA
5,AA00,SA,202003,D,15,14,D,11,0905,1,...,40,F,2,1,149621,05.3,2020-03,2020,3,AA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717108,ZU00,SA,202111,K,17,,E,11,0301,2,...,48,F,2,1,170800,14.5,2021-11,2021,11,ZU
717109,ZU00,SJ,202107,E,17,,D,11,0301,2,...,38,F,1,1,112930,08.1,2021-07,2021,7,ZU
717110,ZU00,SA,202103,K,13,,D,11,0301,2,...,38,F,1,1,133465,08.0,2021-03,2021,3,ZU
717111,ZU00,SJ,202107,C,13,,B,11,0301,2,...,38,F,1,1,064682,02.8,2021-07,2021,7,ZU


In [54]:
agency = separations_table.groupby('AGENCY')

In [55]:
agency

AGENCY
AA         9
AB        39
AF     67120
AG     52325
AH       206
AM       995
AN        28
AP         6
AR     89620
AU        40
AW        20
BD        64
BF        26
BG       297
BH        10
BK         2
BO       318
BT        14
BW         4
CC        20
CE        42
CF         1
CM     28658
CT       178
CU       253
DB         3
DD     44884
DG         3
DJ     26689
DL      3823
DN      3983
DQ         5
EB       173
EC       152
ED      1123
EE       636
EO        12
EP      3552
EQ        14
EW        25
FC       316
FD      1770
FI         2
FJ        22
FK         2
FL       113
FM        42
FQ       193
FR       379
FT       366
FW        50
GB       174
GC         5
GE         1
GG        13
GJ       160
GQ        46
GS      2740
GU         1
GW        80
GX        30
GY         6
GZ         9
HB        16
HD        31
HE     21175
HF       227
HP        15
HS     50816
HT         3
HU      2097
HW        13
IB       284
IF        24
IG        20
IN     40429
IP   

In [56]:
agency_sep_type = separations_table.groupby('AGENCY')['SEP'].sum()

In [57]:
agency_sep_type 

AGENCY
AA                                   SJSCSCSASASASCSASC
AB    SJSASDSDSASCSDSCSDSDSDSDSDSDSDSDSDSASASASASDSD...
AF    SDSASASDSDSDSCSDSASDSDSDSDSDSDSASFSDSCSCSCSDSC...
AG    SASDSDSCSCSCSCSASASCSASDSCSASCSCSCSASCSCSASASC...
AH    SCSJSDSDSASDSASASCSDSDSDSDSASDSDSDSDSCSCSCSJSA...
AM    SDSASASCSCSCSASCSCSCSCSCSCSKSGSDSCSDSCSCSDSDSD...
AN    SASDSDSDSCSJSKSCSASJSCSASHSASDSASASCSASASASASC...
AP                                         SCSJSDSCSCSC
AR    SCSCSASASDSASDSASASDSDSASDSDSCSASDSDSASDSASASD...
AU    SASDSASCSDSCSDSDSDSCSCSDSJSCSCSCSCSASCSCSASCSA...
AW             SJSDSJSJSJSJSJSCSJSKSJSJSJSJSJSJSJSJSJSJ
BD    SASKSDSASASDSCSDSASDSDSDSDSASDSDSDSASDSCSDSASD...
BF    SASASASDSDSKSDSASASCSASDSDSCSASDSCSDSASASCSASJ...
BG    SASDSDSASJSASCSDSASASDSDSDSJSDSKSKSKSDSDSDSDSA...
BH                                 SCSJSCSJSJSJSJSJSJSJ
BK                                                 SDSC
BO    SCSDSASCSASCSASCSDSDSCSASCSASASDSASDSCSDSCSDSA...
BT                         SASDSASDSDSDSJ